In [1]:
import pandas as pd

In [2]:
import numpy as np
import re

In [3]:
import nltk

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from textblob import TextBlob
import emoji

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [6]:
file_path = "/home/kaushalpancholi/Documents/6000+ Indian Food Recipes Dataset/IndianFoodDatasetCSV.csv"
df = pd.read_csv(file_path)


In [7]:
print(df)

       Srno                                         RecipeName  \
0         1                               Masala Karela Recipe   
1         2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2         3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3         4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4         5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   
...     ...                                                ...   
6866  14073  गोअन मशरुम जकुटी रेसिपी - Goan Mushroom Xacuti...   
6867  14107  शकरकंदी और मेथी का पराठा रेसिपी - Sweet Potato...   
6868  14165      Ullikadala Pulusu Recipe | Spring Onion Curry   
6869  14167  Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...   
6870  14211              नवरंग दाल रेसिपी - Navrang Dal Recipe   

                                   TranslatedRecipeName  \
0                                  Masala Karela Recipe   
1                            Spicy Tomato Rice (Recipe)   
2     Ragi Semiya Upma Recipe 

In [8]:
print(list(df.columns))

['Srno', 'RecipeName', 'TranslatedRecipeName', 'Ingredients', 'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins', 'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet', 'Instructions', 'TranslatedInstructions', 'URL']


In [9]:
print(df.head())

   Srno                                         RecipeName  \
0     1                               Masala Karela Recipe   
1     2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2     3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3     4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4     5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   

                                TranslatedRecipeName  \
0                               Masala Karela Recipe   
1                         Spicy Tomato Rice (Recipe)   
2  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3  Gongura Chicken Curry Recipe - Andhra Style Go...   
4  Andhra Style Alam Pachadi Recipe - Adrak Chutn...   

                                         Ingredients  \
0  6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...   
1  2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...   
2  1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...   
3  500 grams Chicken,2 Onion - chopped,1 Tomato -...   
4  1 बड़ा च

In [12]:
# def preprocess_text(text):
#     # Tokenization
#     tokens = word_tokenize(text)
    
#     # Convert to lowercase
#     tokens = [token.lower() for token in tokens]
    
#     # Remove punctuation
#     tokens = [token for token in tokens if token.isalpha()]
    
#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = [token for token in tokens if token not in stop_words]
    
#     # Lemmatization
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
#     return ' '.join(tokens)


In [13]:
# df['processed_recipe'] = df['TranslatedRecipeName'].apply(preprocess_text)
# print(df[['TranslatedRecipeName', 'processed_recipe']].head())

In [12]:
# Select the columns to preprocess
columns_to_preprocess = ['Ingredients', 'TranslatedIngredients']

In [19]:
# Function to clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    # text = emoji.get_emoji_regexp().sub(u'', text)  # Remove emojis
    return text

In [20]:
for column in columns_to_preprocess:
    df[column + '_clean'] = df[column].astype(str).apply(clean_text)
    print(f"Cleaned data for {column}:")
    print(df[column + '_clean'].head())
    print("\n")

Cleaned data for Ingredients:
0     karela bitter gourd pavakkai  deseededsalt  t...
1     कप चवल  पक ल टमटर छट चमचच बस बल भत पउडरनमक  स...
2     cups rice vermicelli noodles thin onion  slic...
3     grams chicken onion  chopped tomato  chopped ...
4     बड़ चमचच चन दल बड़ चमचच सफ़द उरद दल सख लल मरच बड़...
Name: Ingredients_clean, dtype: object


Cleaned data for TranslatedIngredients:
0     karela bitter gourd pavakkai  deseededsalt  t...
1       cups rice  cooked  tomatoes  teaspoons bc b...
2     cups rice vermicelli noodles thin onion  slic...
3     grams chicken onion  chopped tomato  chopped ...
4     tablespoon chana dal  tablespoon white urad d...
Name: TranslatedIngredients_clean, dtype: object




In [21]:
# Function to tokenize text
def tokenize(text):
    return word_tokenize(text)

In [22]:
for column in columns_to_preprocess:
    df[column + '_tokens'] = df[column + '_clean'].apply(tokenize)
    print(f"Tokenized data for {column}:")
    print(df[column + '_tokens'].head())
    print("\n")

Tokenized data for Ingredients:
0    [karela, bitter, gourd, pavakkai, deseededsalt...
1    [कप, चवल, पक, ल, टमटर, छट, चमचच, बस, बल, भत, प...
2    [cups, rice, vermicelli, noodles, thin, onion,...
3    [grams, chicken, onion, chopped, tomato, chopp...
4    [बड़, चमचच, चन, दल, बड़, चमचच, सफ़द, उरद, दल, सख,...
Name: Ingredients_tokens, dtype: object


Tokenized data for TranslatedIngredients:
0    [karela, bitter, gourd, pavakkai, deseededsalt...
1    [cups, rice, cooked, tomatoes, teaspoons, bc, ...
2    [cups, rice, vermicelli, noodles, thin, onion,...
3    [grams, chicken, onion, chopped, tomato, chopp...
4    [tablespoon, chana, dal, tablespoon, white, ur...
Name: TranslatedIngredients_tokens, dtype: object




In [23]:
# Function to remove stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if not token in stop_words]

In [24]:
for column in columns_to_preprocess:
    df[column + '_no_stopwords'] = df[column + '_tokens'].apply(remove_stopwords)
    print(f"Data without stopwords for {column}:")
    print(df[column + '_no_stopwords'].head())
    print("\n")

Data without stopwords for Ingredients:
0    [karela, bitter, gourd, pavakkai, deseededsalt...
1    [कप, चवल, पक, ल, टमटर, छट, चमचच, बस, बल, भत, प...
2    [cups, rice, vermicelli, noodles, thin, onion,...
3    [grams, chicken, onion, chopped, tomato, chopp...
4    [बड़, चमचच, चन, दल, बड़, चमचच, सफ़द, उरद, दल, सख,...
Name: Ingredients_no_stopwords, dtype: object


Data without stopwords for TranslatedIngredients:
0    [karela, bitter, gourd, pavakkai, deseededsalt...
1    [cups, rice, cooked, tomatoes, teaspoons, bc, ...
2    [cups, rice, vermicelli, noodles, thin, onion,...
3    [grams, chicken, onion, chopped, tomato, chopp...
4    [tablespoon, chana, dal, tablespoon, white, ur...
Name: TranslatedIngredients_no_stopwords, dtype: object




In [25]:
# Function to perform lemmatization
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

In [26]:
for column in columns_to_preprocess:
    df[column + '_lemmatized'] = df[column + '_no_stopwords'].apply(lemmatize)
    print(f"Lemmatized data for {column}:")
    print(df[column + '_lemmatized'].head())
    print("\n")

Lemmatized data for Ingredients:
0    [karela, bitter, gourd, pavakkai, deseededsalt...
1    [कप, चवल, पक, ल, टमटर, छट, चमचच, बस, बल, भत, प...
2    [cup, rice, vermicelli, noodle, thin, onion, s...
3    [gram, chicken, onion, chopped, tomato, choppe...
4    [बड़, चमचच, चन, दल, बड़, चमचच, सफ़द, उरद, दल, सख,...
Name: Ingredients_lemmatized, dtype: object


Lemmatized data for TranslatedIngredients:
0    [karela, bitter, gourd, pavakkai, deseededsalt...
1    [cup, rice, cooked, tomato, teaspoon, bc, bell...
2    [cup, rice, vermicelli, noodle, thin, onion, s...
3    [gram, chicken, onion, chopped, tomato, choppe...
4    [tablespoon, chana, dal, tablespoon, white, ur...
Name: TranslatedIngredients_lemmatized, dtype: object




In [27]:
# Function to perform POS tagging
def pos_tagging(tokens):
    return pos_tag(tokens)

In [28]:
for column in columns_to_preprocess:
    df[column + '_pos_tags'] = df[column + '_lemmatized'].apply(pos_tagging)
    print(f"POS tagged data for {column}:")
    print(df[column + '_pos_tags'].head())
    print("\n")

POS tagged data for Ingredients:
0    [(karela, NN), (bitter, NN), (gourd, NN), (pav...
1    [(कप, JJ), (चवल, NNP), (पक, NNP), (ल, NNP), (ट...
2    [(cup, NN), (rice, NN), (vermicelli, NN), (noo...
3    [(gram, NN), (chicken, NN), (onion, NN), (chop...
4    [(बड़, JJ), (चमचच, NNP), (चन, NNP), (दल, NNP), ...
Name: Ingredients_pos_tags, dtype: object


POS tagged data for TranslatedIngredients:
0    [(karela, NN), (bitter, NN), (gourd, NN), (pav...
1    [(cup, NN), (rice, NN), (cooked, VBD), (tomato...
2    [(cup, NN), (rice, NN), (vermicelli, NN), (noo...
3    [(gram, NN), (chicken, NN), (onion, NN), (chop...
4    [(tablespoon, NN), (chana, NN), (dal, JJ), (ta...
Name: TranslatedIngredients_pos_tags, dtype: object




In [29]:
# Function for named entity recognition
def named_entity_recognition(pos_tags):
    tree = conlltags2tree(tree2conlltags(ne_chunk(pos_tags)))
    return tree

In [30]:
for column in columns_to_preprocess:
    df[column + '_ner'] = df[column + '_pos_tags'].apply(named_entity_recognition)
    print(f"Named entities for {column}:")
    print(df[column + '_ner'].head())
    print("\n")

Named entities for Ingredients:
0    [(karela, NN), (bitter, NN), (gourd, NN), (pav...
1    [(कप, JJ), [(चवल, NNP)], (पक, NNP), (ल, NNP), ...
2    [(cup, NN), (rice, NN), (vermicelli, NN), (noo...
3    [(gram, NN), (chicken, NN), (onion, NN), (chop...
4    [(बड़, JJ), [(चमचच, NNP)], (चन, NNP), (दल, NNP)...
Name: Ingredients_ner, dtype: object


Named entities for TranslatedIngredients:
0    [(karela, NN), (bitter, NN), (gourd, NN), (pav...
1    [(cup, NN), (rice, NN), (cooked, VBD), (tomato...
2    [(cup, NN), (rice, NN), (vermicelli, NN), (noo...
3    [(gram, NN), (chicken, NN), (onion, NN), (chop...
4    [(tablespoon, NN), (chana, NN), (dal, JJ), (ta...
Name: TranslatedIngredients_ner, dtype: object




In [31]:
# Bag of Words Vectorization
count_vectorizer = CountVectorizer()
df['Ingredients_bow'] = list(count_vectorizer.fit_transform(df['Ingredients_lemmatized'].apply(lambda x: ' '.join(x))).toarray())
print("Bag of Words vectors for Ingredients:")
print(df['Ingredients_bow'].head())

Bag of Words vectors for Ingredients:
0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Ingredients_bow, dtype: object


In [32]:
# Word2Vec Embedding
word2vec_model = Word2Vec(df['Ingredients_lemmatized'], vector_size=100, window=5, min_count=1, workers=4)
df['Ingredients_word2vec'] = df['Ingredients_lemmatized'].apply(lambda x: np.mean([word2vec_model.wv[token] for token in x if token in word2vec_model.wv], axis=0))
print("Word2Vec vectors for Ingredients:")
print(df['Ingredients_word2vec'].head())

Word2Vec vectors for Ingredients:
0    [-1.3646272, 0.6158518, -1.1022518, -0.4020760...
1    [1.4745777, -0.13950716, 1.0738394, 0.05996406...
2    [-1.304461, 0.61397624, -1.0359329, -0.3339517...
3    [-1.36038, 0.6368147, -1.2757882, -0.3278955, ...
4    [1.3943256, -0.19423814, 1.0712048, 0.2121522,...
Name: Ingredients_word2vec, dtype: object


In [33]:
# # Example of using Word2Vec for embeddings
# word2vec_model = Word2Vec(df['Ingredients_lemmatized'], min_count=1)

In [34]:
# View the preprocessed data
# print(df.head())  # Displays the first few rows of the DataFrame